# Introduction

In this notebook, we want to study the two datasets in `beerAdv_beer_brewery.tsv` and in `rateBeer_beer_brewery.tsv`. The columns of these datasets are like this:

            beer_id ¦ beer_name ¦ brewery_id ¦ brewery_name
            
The idea of this notebook is to create a similarity measure in order to find the same elements in both these datasets. Once we have this similarity measure, we want to find the limit X such that:
- All pairs with a similarity value of $X' > X$ are considered as the same beer
- All pairs with a similarity value of $X'' < X$ are not taken into account.

In order to achieve this, this notebook is composed of three parts. The first part is some sort of simple Data Analysis. Then there will be the creation of the pairs and the similarity value. Finally the part to find the best value $X$.

In [1]:
# Usefule imports
import pandas as pd
import numpy as np

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Data Analysis

First, we need to analyze the data. It's a simple data analysis since we only have two usefull features: `beer_name` and `brewery_name`.

In [2]:
# Datasets
beerAdvisor_dataset = './data/beerAdv_beer_brewery.tsv'
rateBeer_dataset = './data/rateBeer_beer_brewery.tsv'

In [3]:
columns = ['beer_id', 'beer_name', 'brewery_id', 'brewery_name']

In [4]:
# Load the dataset with pandas
beerAdvisor = pd.read_table(beerAdvisor_dataset, header=None)
beerAdvisor.columns = columns
rateBeer = pd.read_table(rateBeer_dataset, header=None)
rateBeer.columns = columns

Print the two datasets

In [5]:
beerAdvisor.head()

,beer_id,beer_name,brewery_id,brewery_name
0,14348,Eisbrau Czech,1,"Plzensky Prazdroj, a. s."
1,19099,Primus,1,"Plzensky Prazdroj, a. s."
2,19123,Gambrinus Pale,1,"Plzensky Prazdroj, a. s."
3,19274,Urutislav,1,"Plzensky Prazdroj, a. s."
4,41294,Pilsner Urquell 3.5%,1,"Plzensky Prazdroj, a. s."


In [6]:
rateBeer.head()

,beer_id,beer_name,brewery_id,brewery_name
0,4,Abita Bock,1,Abita Brewing Company
1,10731,Abita Louisiana Red Ale,1,Abita Brewing Company
2,114065,Abita Select Pecan Brown Ale,1,Abita Brewing Company
3,114981,Abita Select Amber Ale,1,Abita Brewing Company
4,117017,Abita American Wheat,1,Abita Brewing Company


We know that the ids will not be the same in both datasets. Therefore, we can just drop the columns.

In [7]:
beerAdvisor = beerAdvisor.drop(['beer_id', 'brewery_id'], axis=1)
rateBeer = rateBeer.drop(['beer_id', 'brewery_id'], axis=1)

Let's print the number of entries in each dataset.

In [8]:
print("Number of rows in beerAdvisor: %i"%(len(beerAdvisor)))
print("Number of rows in rateBeer: %i"%(len(rateBeer)))

Number of rows in beerAdvisor: 66056
Number of rows in rateBeer: 110359


Let's check the number of **unique** beers to see if it matches the number of rows in the dataset.

In [9]:
unique_beers_beerAdvisor = beerAdvisor.beer_name.unique()
unique_beers_rateBeer = rateBeer.beer_name.unique()

print("Number of unique beers in beerAdvisor: %i"%(len(unique_beers_beerAdvisor)))
print("Number of unique beers in rateBeer: %i"%(len(unique_beers_rateBeer)))

Number of unique beers in beerAdvisor: 56855
Number of unique beers in rateBeer: 110302


It's already interesting to see that some beers have the same name in both datasets. Let's take one of them and see if they have the same `brewery_name`.

In [10]:
duplicated_beers_beerAdvisor = beerAdvisor[beerAdvisor.beer_name.duplicated()].beer_name.unique()
duplicated_beers_rateBeer = rateBeer[rateBeer.beer_name.duplicated()].beer_name.unique()

print("Number of duplicated beer names in beerAdvisor: %i"%(len(duplicated_beers_beerAdvisor)))
print("Number of duplicated beer names in rateBeer: %i"%(len(duplicated_beers_rateBeer)))

Number of duplicated beer names in beerAdvisor: 2707
Number of duplicated beer names in rateBeer: 55


In [11]:
dup_beer_dup_brewery = []

# Create list of tuples. 
#   First entry is the name of the duplicated beer
#   Second entry is the list of Brewery that is duplicated
#   Third entry is the list of indices to remove them easily.
for dup_beer in duplicated_beers_beerAdvisor:
    subdf = beerAdvisor[beerAdvisor.beer_name == dup_beer]
    if any(subdf.brewery_name.duplicated()):
        dup_beer_dup_brewery.append((dup_beer, list(subdf[subdf.brewery_name.duplicated()]["brewery_name"]), list(subdf[subdf.brewery_name.duplicated()].index)))

In [12]:
print("Number of beers with duplicated brewery in beerAdvisor: %i"%(len(dup_beer_dup_brewery)))

Number of beers with duplicated brewery in beerAdvisor: 286


Just check how the function `duplicated` works. For example, if we have two times the same brewery, this means that we would have 3 times the same beer and the same brewery in the data set.

In [13]:
dup_beer_dup_brewery[3]

('Saison',
 ['Triumph Brewing Company', 'Triumph Brewing Company'],
 [22563, 43875])

In [14]:
beerAdvisor[(beerAdvisor.beer_name == "Saison") & (beerAdvisor.brewery_name == "Triumph Brewing Company")]

,beer_name,brewery_name
17881,Saison,Triumph Brewing Company
22563,Saison,Triumph Brewing Company
43875,Saison,Triumph Brewing Company


Verfied. Therefore, we can just remove the indices.

In [15]:
indices_to_remove = []
for i in range(len(dup_beer_dup_brewery)):
    indices_to_remove.extend(dup_beer_dup_brewery[i][2])

In [16]:
# Remove the indices in the dataset
beerAdvisor = beerAdvisor.drop(indices_to_remove, axis=0)
beerAdvisor.index = range(len(beerAdvisor))
beerAdvisor.tail()

,beer_name,brewery_name
65668,Foster's Gold,Foster's Group Limited
65669,Foster's Special Bitter,Foster's Group Limited
65670,Foster's Lager,Foster's Group Limited
65671,Hefe Weissbier,Bayerische Löwenbrauerei
65672,Hefeweissbier Dunkel,Bayerische Löwenbrauerei


Now, we need to the same for the rateBeer dataset.

In [17]:
dup_beer_dup_brewery = []

# Create list of tuples. 
#   First entry is the name of the duplicated beer
#   Second entry is the list of Brewery that is duplicated
#   Third entry is the list of indices to remove them easily.
for dup_beer in duplicated_beers_rateBeer:
    subdf = rateBeer[rateBeer.beer_name == dup_beer]
    if any(subdf.brewery_name.duplicated()):
        dup_beer_dup_brewery.append((dup_beer, list(subdf[subdf.brewery_name.duplicated()]["brewery_name"]), list(subdf[subdf.brewery_name.duplicated()].index)))

In [18]:
print("Number of beers with duplicated brewery in beerRate: %i"%(len(dup_beer_dup_brewery)))

Number of beers with duplicated brewery in beerRate: 4


In [19]:
dup_beer_dup_brewery

[('Traugott Simon Export', ['Udo Täubrich Betreuungs'], [9819]),
 ('Big Horn Saison',
  ['Big Horn Brewing Company (Ram International)'],
  [19577]),
 ('Yukon Lead Dog Ale', ['Yukon Brewing Company'], [37365]),
 ('Prison Brews Winter Ale', ['Prison Brews'], [77632])]

In [20]:
indices_to_remove = []
for i in range(len(dup_beer_dup_brewery)):
    indices_to_remove.extend(dup_beer_dup_brewery[i][2])

In [21]:
# Remove the indices in the dataset
rateBeer = rateBeer.drop(indices_to_remove, axis=0)
rateBeer.index = range(len(rateBeer))
rateBeer.tail()

,beer_name,brewery_name
110350,Jelling Bryghus Frode Fredegod,Jelling Bryghus
110351,Jelling Bryghus Poppo,Jelling Bryghus
110352,Jelling Bryghus Jelling Jól,Jelling Bryghus
110353,Jelling Bryghus Jalunki,Jelling Bryghus
110354,Jelling Bryghus Sildeglimt,Jelling Bryghus


### Save the cleaned data set

Now that the datasets are cleaned, we can save them in a CSV format.

In [22]:
beerAdvisor.to_csv('./data/beerAdvisor_cleaned.csv', index=False)
rateBeer.to_csv('./data/rateBeer_cleaned.csv', index=False)

# Similarity

** YOU CAN DIRECTLY START FROM HERE. NO NEED TO REDO THE CLEANING OF THE DATASETS! **

Now, we want to match beers between the two different datasets. TO BE CONTINUED...

In [23]:
beerAdvisor = pd.read_csv('./data/beerAdvisor_cleaned.csv')
rateBeer = pd.read_csv('./data/rateBeer_cleaned.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 13: invalid continuation byte

In [ ]:
£